In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/raw/202008-divvy-tripdata.csv")

In [3]:
len(df)

622361

In [4]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,322BD23D287743ED,docked_bike,2020-08-20 18:08:14,2020-08-20 18:17:51,Lake Shore Dr & Diversey Pkwy,329.0,Clark St & Lincoln Ave,141.0,41.932588,-87.636427,41.915689,-87.634600,member
1,2A3AEF1AB9054D8B,electric_bike,2020-08-27 18:46:04,2020-08-27 19:54:51,Michigan Ave & 14th St,168.0,Michigan Ave & 14th St,168.0,41.864379,-87.623681,41.864221,-87.623439,casual
2,67DC1D133E8B5816,electric_bike,2020-08-26 19:44:14,2020-08-26 21:53:07,Columbus Dr & Randolph St,195.0,State St & Randolph St,44.0,41.884641,-87.619549,41.884971,-87.627574,casual
3,C79FBBD412E578A7,electric_bike,2020-08-27 12:05:41,2020-08-27 12:53:45,Daley Center Plaza,81.0,State St & Kinzie St,47.0,41.884093,-87.629639,41.889583,-87.627540,casual
4,13814D3D661ECADB,electric_bike,2020-08-27 16:49:02,2020-08-27 16:59:49,Leavitt St & Division St,658.0,Leavitt St & Division St,658.0,41.902989,-87.683767,41.903002,-87.683844,casual


In [5]:
df.describe()

,start_station_id,end_station_id,start_lat,start_lng,end_lat,end_lng
count,614670.000000,612251.000000,622361.000000,622361.000000,621423.000000,621423.000000
mean,225.413108,225.801495,41.905512,-87.644040,41.905778,-87.644252
std,161.761515,161.568614,0.041991,0.025201,0.042113,0.025328
min,2.000000,2.000000,41.660000,-87.870000,41.660000,-87.890000
25%,96.000000,97.000000,41.883521,-87.656936,41.883668,-87.658040
50%,196.000000,196.000000,41.902308,-87.640000,41.902861,-87.641066
75%,312.000000,312.000000,41.931248,-87.626804,41.931320,-87.626819
max,700.000000,700.000000,42.070000,-87.530000,42.160000,-87.530000


In [6]:
df.isna().sum()

ride_id                   0
rideable_type             0
started_at                0
ended_at                  0
start_station_name     7595
start_station_id       7691
end_station_name      10035
end_station_id        10110
start_lat                 0
start_lng                 0
end_lat                 938
end_lng                 938
member_casual             0
dtype: int64

In [7]:
(df.isna().sum() / len(df))*100

ride_id               0.000000
rideable_type         0.000000
started_at            0.000000
ended_at              0.000000
start_station_name    1.220353
start_station_id      1.235778
end_station_name      1.612408
end_station_id        1.624459
start_lat             0.000000
start_lng             0.000000
end_lat               0.150716
end_lng               0.150716
member_casual         0.000000
dtype: float64

In [8]:
df_nulls = df.isna().sum()
df_nulls[df_nulls > 0].sort_values(ascending = False)

end_station_id        10110
end_station_name      10035
start_station_id       7691
start_station_name     7595
end_lat                 938
end_lng                 938
dtype: int64

Faz sentido ter valores nulos?
de acordo com a descrição do problema as bicicletas só podem ser coletadas em um posto de bicicletas e entregues lá, portanto não há interpretação em relação à valores ausentes.
Sabendo disso , basta removermos os valore nulos

No entanto vale a pena anlisar Lat e Long , visto que pode ser que o nome da estação esteja ausente , mas a posição dele "entrega" a estação

### Vamos tentar usar a lat e long para encontrar o pontos

In [9]:
cols_coord = ['start_lat', 'start_lng', 'end_lat', 'end_lng']
mask_nulls = df.isna().any(axis=1)

df_nulls = df[mask_nulls]
df_coords_mantain = df.dropna(subset=cols_coord)
df_not_null = df[~mask_nulls]

In [10]:
df_nulls.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
15,EC02009ADA98F964,electric_bike,2020-08-26 22:07:46,2020-08-26 22:14:37,Dearborn St & Monroe St,49.0,NaN,NaN,41.880876,-87.629690,41.890000,-87.630000,casual
16,CBE132DC5360F780,electric_bike,2020-08-27 09:07:44,2020-08-27 09:32:45,NaN,NaN,Clinton St & Roosevelt Rd,57.0,41.850000,-87.690000,41.867185,-87.641049,casual
17,C106B2A1A15BF053,electric_bike,2020-08-27 18:58:47,2020-08-27 20:18:24,NaN,NaN,NaN,NaN,41.870000,-87.610000,41.850000,-87.620000,casual
18,C65DBF5DBF4D595A,electric_bike,2020-08-26 22:24:59,2020-08-26 23:39:38,NaN,NaN,NaN,NaN,41.890000,-87.630000,41.850000,-87.690000,casual
19,2D761BDE43A34769,electric_bike,2020-08-27 17:34:41,2020-08-27 17:59:25,Indiana Ave & Roosevelt Rd,255.0,NaN,NaN,41.867933,-87.623027,41.870000,-87.610000,casual


In [11]:
df_not_null.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,322BD23D287743ED,docked_bike,2020-08-20 18:08:14,2020-08-20 18:17:51,Lake Shore Dr & Diversey Pkwy,329.0,Clark St & Lincoln Ave,141.0,41.932588,-87.636427,41.915689,-87.634600,member
1,2A3AEF1AB9054D8B,electric_bike,2020-08-27 18:46:04,2020-08-27 19:54:51,Michigan Ave & 14th St,168.0,Michigan Ave & 14th St,168.0,41.864379,-87.623681,41.864221,-87.623439,casual
2,67DC1D133E8B5816,electric_bike,2020-08-26 19:44:14,2020-08-26 21:53:07,Columbus Dr & Randolph St,195.0,State St & Randolph St,44.0,41.884641,-87.619549,41.884971,-87.627574,casual
3,C79FBBD412E578A7,electric_bike,2020-08-27 12:05:41,2020-08-27 12:53:45,Daley Center Plaza,81.0,State St & Kinzie St,47.0,41.884093,-87.629639,41.889583,-87.627540,casual
4,13814D3D661ECADB,electric_bike,2020-08-27 16:49:02,2020-08-27 16:59:49,Leavitt St & Division St,658.0,Leavitt St & Division St,658.0,41.902989,-87.683767,41.903002,-87.683844,casual


In [12]:
df_coords_mantain.head(10)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,322BD23D287743ED,docked_bike,2020-08-20 18:08:14,2020-08-20 18:17:51,Lake Shore Dr & Diversey Pkwy,329.0,Clark St & Lincoln Ave,141.0,41.932588,-87.636427,41.915689,-87.634600,member
1,2A3AEF1AB9054D8B,electric_bike,2020-08-27 18:46:04,2020-08-27 19:54:51,Michigan Ave & 14th St,168.0,Michigan Ave & 14th St,168.0,41.864379,-87.623681,41.864221,-87.623439,casual
2,67DC1D133E8B5816,electric_bike,2020-08-26 19:44:14,2020-08-26 21:53:07,Columbus Dr & Randolph St,195.0,State St & Randolph St,44.0,41.884641,-87.619549,41.884971,-87.627574,casual
3,C79FBBD412E578A7,electric_bike,2020-08-27 12:05:41,2020-08-27 12:53:45,Daley Center Plaza,81.0,State St & Kinzie St,47.0,41.884093,-87.629639,41.889583,-87.627540,casual
4,13814D3D661ECADB,electric_bike,2020-08-27 16:49:02,2020-08-27 16:59:49,Leavitt St & Division St,658.0,Leavitt St & Division St,658.0,41.902989,-87.683767,41.903002,-87.683844,casual
5,56349A5A42F0AE51,electric_bike,2020-08-27 17:26:23,2020-08-27 18:07:50,Leavitt St & Division St,658.0,Leavitt St & Division St,658.0,41.903015,-87.683731,41.903092,-87.683628,casual
6,EB6ABC5570C29B22,electric_bike,2020-08-26 20:14:02,2020-08-26 20:34:00,Cityfront Plaza Dr & Pioneer Ct,196.0,Dearborn St & Monroe St,49.0,41.890326,-87.621700,41.880626,-87.629343,casual
7,B4ECE389A1DE922D,electric_bike,2020-08-26 21:59:50,2020-08-26 22:12:35,Sheffield Ave & Fullerton Ave,67.0,Southport Ave & Roscoe St,229.0,41.925546,-87.653692,41.943745,-87.664065,casual
8,0B355B0FE076D010,electric_bike,2020-08-26 19:17:42,2020-08-26 19:32:14,Southport Ave & Wellington Ave,153.0,Halsted St & Dickens Ave,225.0,41.935729,-87.663966,41.919987,-87.648989,casual
9,1ECE04F779E9FDF6,electric_bike,2020-08-27 15:13:57,2020-08-27 15:41:59,Theater on the Lake,177.0,Western Ave & Division St,305.0,41.926270,-87.630955,41.902911,-87.687201,casual


In [13]:
qt_removed_lines = len(df_nulls)
percent_removed_lines = (len(df_nulls)/len(df_not_null))*100
qt_lines_coords_exists = len(df_coords_mantain) - len(df)

print(f"We removed {qt_removed_lines} lines , which represents {percent_removed_lines}% of our dataset")
print(f"But , we have {qt_lines_coords_exists} lines that have Lat and Long , but don't have Station name (start or end)")

We removed 13851 lines , which represents 2.276215674352106% of our dataset
But , we have -938 lines that have Lat and Long , but don't have Station name (start or end)


#### 

Sabendo dessa informação , vamos separar em um df a parte apenas as estações

In [14]:
df_stations = df_not_null[["start_station_name", "start_station_id" , "end_station_name" , "end_station_id" , "start_lat" , "start_lng" , "end_lat" , "end_lng"]]
df_start_station = df_stations[["start_station_name" , "start_station_id" , "start_lat" , "start_lng"]]
df_end_station = df_stations[["end_station_name" , "end_station_id" , "end_lat" , "end_lng"]]

In [15]:
df_start_station = df_start_station.drop_duplicates(subset="start_station_id")
df_start_station.columns = ["station" , "id" , "lat" , "long"]
df_start_station.head()

,station,id,lat,long
0,Lake Shore Dr & Diversey Pkwy,329.0,41.932588,-87.636427
1,Michigan Ave & 14th St,168.0,41.864379,-87.623681
2,Columbus Dr & Randolph St,195.0,41.884641,-87.619549
3,Daley Center Plaza,81.0,41.884093,-87.629639
4,Leavitt St & Division St,658.0,41.902989,-87.683767


In [16]:
df_end_station = df_end_station.drop_duplicates(subset="end_station_id")
df_end_station.columns = ["station" , "id" , "lat" , "long"]
df_end_station.head()

,station,id,lat,long
0,Clark St & Lincoln Ave,141.0,41.915689,-87.634600
1,Michigan Ave & 14th St,168.0,41.864221,-87.623439
2,State St & Randolph St,44.0,41.884971,-87.627574
3,State St & Kinzie St,47.0,41.889583,-87.627540
4,Leavitt St & Division St,658.0,41.903002,-87.683844


In [17]:
print(f"stations start = {len(df_start_station)} and stations end = {len(df_end_station)}")

stations start = 633 and stations end = 633


In [18]:
frames = [df_start_station , df_end_station]
df_stations = pd.concat(frames)
len(df_stations)

1266

In [19]:
df_stations.head()

,station,id,lat,long
0,Lake Shore Dr & Diversey Pkwy,329.0,41.932588,-87.636427
1,Michigan Ave & 14th St,168.0,41.864379,-87.623681
2,Columbus Dr & Randolph St,195.0,41.884641,-87.619549
3,Daley Center Plaza,81.0,41.884093,-87.629639
4,Leavitt St & Division St,658.0,41.902989,-87.683767


In [20]:
df_stations = df_stations.drop_duplicates(subset="id")

In [21]:
decimal = 4

df_stations["lat"] = df_stations["lat"].round(decimal)
df_stations["long"] = df_stations["long"].round(decimal)

In [22]:
# Essa é minha "DIM_STATIONS"
df_stations.head()

,station,id,lat,long
0,Lake Shore Dr & Diversey Pkwy,329.0,41.9326,-87.6364
1,Michigan Ave & 14th St,168.0,41.8644,-87.6237
2,Columbus Dr & Randolph St,195.0,41.8846,-87.6195
3,Daley Center Plaza,81.0,41.8841,-87.6296
4,Leavitt St & Division St,658.0,41.9030,-87.6838


In [23]:
decimal = 4
df_coords_mantain = df_coords_mantain[["ride_id" , "rideable_type" , "started_at" , "ended_at" , "start_lat" , "start_lng" , "end_lat" , "end_lng"]]
df_coords_mantain['start_lat_arred'] = df_coords_mantain['start_lat'].round(decimal)
df_coords_mantain['start_lng_arred'] = df_coords_mantain['start_lng'].round(decimal)
df_coords_mantain['end_lat_arred'] = df_coords_mantain['end_lat'].round(decimal)
df_coords_mantain['end_lng_arred'] = df_coords_mantain['end_lng'].round(decimal)

In [24]:
df_resultado = pd.merge(
    df_coords_mantain, 
    df_stations,
    # Usamos as colunas arredondadas como chave
    left_on=['start_lat_arred', 'start_lng_arred'],
    right_on=['lat', 'long'],
    how='left',
    suffixes=('', '_start') # Para diferenciar colunas com mesmo nome
)

In [25]:
df_resultado = pd.merge(
    df_resultado, 
    df_stations,
    # Usamos as colunas arredondadas como chave
    left_on=['end_lat_arred', 'end_lng_arred'],
    right_on=['lat', 'long'],
    how='left',
    suffixes=('', '_end') # Para diferenciar colunas com mesmo nome
)

In [26]:
df_resultado.head()

,ride_id,rideable_type,started_at,ended_at,start_lat,start_lng,end_lat,end_lng,start_lat_arred,start_lng_arred,end_lat_arred,end_lng_arred,station,id,lat,long,station_end,id_end,lat_end,long_end
0,322BD23D287743ED,docked_bike,2020-08-20 18:08:14,2020-08-20 18:17:51,41.932588,-87.636427,41.915689,-87.634600,41.9326,-87.6364,41.9157,-87.6346,Lake Shore Dr & Diversey Pkwy,329.0,41.9326,-87.6364,Clark St & Lincoln Ave,141.0,41.9157,-87.6346
1,2A3AEF1AB9054D8B,electric_bike,2020-08-27 18:46:04,2020-08-27 19:54:51,41.864379,-87.623681,41.864221,-87.623439,41.8644,-87.6237,41.8642,-87.6234,Michigan Ave & 14th St,168.0,41.8644,-87.6237,NaN,NaN,NaN,NaN
2,67DC1D133E8B5816,electric_bike,2020-08-26 19:44:14,2020-08-26 21:53:07,41.884641,-87.619549,41.884971,-87.627574,41.8846,-87.6195,41.8850,-87.6276,Columbus Dr & Randolph St,195.0,41.8846,-87.6195,NaN,NaN,NaN,NaN
3,C79FBBD412E578A7,electric_bike,2020-08-27 12:05:41,2020-08-27 12:53:45,41.884093,-87.629639,41.889583,-87.627540,41.8841,-87.6296,41.8896,-87.6275,Daley Center Plaza,81.0,41.8841,-87.6296,NaN,NaN,NaN,NaN
4,13814D3D661ECADB,electric_bike,2020-08-27 16:49:02,2020-08-27 16:59:49,41.902989,-87.683767,41.903002,-87.683844,41.9030,-87.6838,41.9030,-87.6838,Leavitt St & Division St,658.0,41.9030,-87.6838,Leavitt St & Division St,658.0,41.9030,-87.6838


In [27]:
len(df_resultado)

621423

In [28]:
df_nulls_temp = df_resultado.isna().sum()
df_nulls_temp.sort_values(ascending = False)

long_end           517283
lat_end            517283
id_end             517283
station_end        517283
station            512073
id                 512073
lat                512073
long               512073
ended_at                0
started_at              0
rideable_type           0
ride_id                 0
end_lng_arred           0
end_lat_arred           0
start_lng_arred         0
start_lat_arred         0
end_lng                 0
end_lat                 0
start_lng               0
start_lat               0
dtype: int64

Usando uma aproximação de 3 casas decimais perdemos mais de 500k linhas de dados apenas para encontrar os pontos mais preciso. O que eu vou tentar agora é considerar apenas um dataframe (este df terá SOMENTE os dados de origin start faltantes)

Não vale a pena tentar fazer esse processo de verificação de lat e long , é melhor só remover dados faltantes

### Vamos apenas usar o df com os dados que precisamos

A ideia aqui é simples. vamos ter 3 dfs (um com ambos os valores de start e end nulos , um apenas com start nulo e outro apenas com end nulo).
Para cada um deles eu farei merges para encontrar as posições de id das estações
Vouy juntar esses 3 dfs em um só e terei o tratamento apenas dos dados faltantes feitos

#### DF Com Start e End nulos

In [29]:
# ambas as colunas são nulas
df_station_start_end_nulls = df[df[["start_station_id","end_station_id"]].isna().all(axis = 1)]

decimal = 4
df_station_start_end_nulls["start_lat_rounded"] = df_station_start_end_nulls['start_lat'].round(decimal)
df_station_start_end_nulls["start_lng_rounded"] = df_station_start_end_nulls['start_lng'].round(decimal)
df_station_start_end_nulls["end_lat_rounded"] = df_station_start_end_nulls['end_lat'].round(decimal)
df_station_start_end_nulls["end_lng_rounded"] = df_station_start_end_nulls['end_lng'].round(decimal)

In [30]:
df_station_start_end_nulls = pd.merge(
    df_station_start_end_nulls, 
    df_stations,
    # Usamos as colunas arredondadas como chave
    left_on=['end_lat_rounded', 'end_lng_rounded'],
    right_on=['lat', 'long'],
    how='left',
    suffixes=('', '_end') # Para diferenciar colunas com mesmo nome
)

In [31]:
df_station_start_end_nulls = pd.merge(
    df_station_start_end_nulls, 
    df_stations,
    # Usamos as colunas arredondadas como chave
    left_on=['start_lat_rounded', 'start_lng_rounded'],
    right_on=['lat', 'long'],
    how='left',
    suffixes=('', '_start_end') # Para diferenciar colunas com mesmo nome
)

In [32]:
len(df_station_start_end_nulls)

3950

In [33]:
df_station_start_end_nulls = df_station_start_end_nulls[df_station_start_end_nulls["id_start_end"].notna() & df_station_start_end_nulls["id"].notna()]

In [34]:
df_station_start_end_nulls.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,end_lat_rounded,end_lng_rounded,station,id,lat,long,station_start_end,id_start_end,lat_start_end,long_start_end


#### DF Com Start nulos

In [35]:
# Apenas start_station_id é nulo
df_station_start_nulls = df[df["start_station_id"].isna() & df['end_station_id'].notna()]

decimal = 4
df_station_start_nulls["start_lat_rounded"] = df_station_start_nulls['start_lat'].round(decimal)
df_station_start_nulls["start_lng_rounded"] = df_station_start_nulls['start_lng'].round(decimal)
df_station_start_nulls["end_lat_rounded"] = df_station_start_nulls['end_lat'].round(decimal)
df_station_start_nulls["end_lng_rounded"] = df_station_start_nulls['end_lng'].round(decimal)

In [36]:
df_station_start_nulls = pd.merge(
    df_station_start_nulls, 
    df_stations,
    # Usamos as colunas arredondadas como chave
    left_on=['end_lat_rounded', 'end_lng_rounded'],
    right_on=['lat', 'long'],
    how='left',
    suffixes=('', '_start') # Para diferenciar colunas com mesmo nome
)

In [37]:
len(df_station_start_nulls)

3741

In [38]:
df_station_start_nulls = df_station_start_nulls[df_station_start_nulls["id"].notna()]

In [39]:
df_station_start_nulls.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,end_lng,member_casual,start_lat_rounded,start_lng_rounded,end_lat_rounded,end_lng_rounded,station,id,lat,long
27,75F20FDAB4AD3A86,electric_bike,2020-08-10 21:43:07,2020-08-10 22:10:06,NaN,NaN,Shore Dr & 55th St,247.0,41.83,-87.60,...,-87.580922,casual,41.83,-87.60,41.7952,-87.5809,Shore Dr & 55th St,247.0,41.7952,-87.5809
34,2A61BE2204BD69B1,electric_bike,2020-08-22 12:00:35,2020-08-22 12:28:17,NaN,NaN,Kedzie Ave & Palmer Ct,290.0,41.92,-87.65,...,-87.707335,casual,41.92,-87.65,41.9215,-87.7073,Kedzie Ave & Palmer Ct,290.0,41.9215,-87.7073
51,DC2912CCFB0ACB5E,electric_bike,2020-08-31 09:04:44,2020-08-31 09:16:16,NaN,NaN,Kedzie Ave & Foster Ave,470.0,41.99,-87.71,...,-87.709274,casual,41.99,-87.71,41.9757,-87.7093,Kedzie Ave & Foster Ave,470.0,41.9757,-87.7093
68,251151100BEAB319,electric_bike,2020-08-20 19:28:23,2020-08-20 20:37:20,NaN,NaN,Milwaukee Ave & Rockwell St,222.0,41.91,-87.71,...,-87.692635,casual,41.91,-87.71,41.9203,-87.6926,Milwaukee Ave & Rockwell St,222.0,41.9203,-87.6926
81,9951CD95EDF79469,electric_bike,2020-08-13 20:04:48,2020-08-13 20:57:02,NaN,NaN,Clark St & Lincoln Ave,141.0,41.89,-87.63,...,-87.634594,casual,41.89,-87.63,41.9157,-87.6346,Clark St & Lincoln Ave,141.0,41.9157,-87.6346


In [40]:
len(df_station_start_nulls)

500

#### DF Com End nulos

In [41]:
# Apenas start_station_id é nulo
df_station_end_nulls = df[df["end_station_id"].isna() & df['start_station_id'].notna()]

decimal = 4
df_station_end_nulls["start_lat_rounded"] = df_station_end_nulls['start_lat'].round(decimal)
df_station_end_nulls["start_lng_rounded"] = df_station_end_nulls['start_lng'].round(decimal)
df_station_end_nulls["end_lat_rounded"] = df_station_end_nulls['end_lat'].round(decimal)
df_station_end_nulls["end_lng_rounded"] = df_station_end_nulls['end_lng'].round(decimal)

In [42]:
len(df_station_end_nulls)

6160

In [43]:
df_station_end_nulls = pd.merge(
    df_station_end_nulls, 
    df_stations,
    # Usamos as colunas arredondadas como chave
    left_on=['end_lat_rounded', 'end_lng_rounded'],
    right_on=['lat', 'long'],
    how='left',
    suffixes=('', '_end') # Para diferenciar colunas com mesmo nome
)

In [44]:
df_station_end_nulls = df_station_end_nulls[df_station_end_nulls["id"].notna()]

In [45]:
df_station_end_nulls.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,end_lng,member_casual,start_lat_rounded,start_lng_rounded,end_lat_rounded,end_lng_rounded,station,id,lat,long


#### Juntando os DF's já removidos os nulos

Primeiro vou organizar as colunas dos df's para ficarem padronizado:

In [46]:
print("Columns necessary:")
for column in df_not_null.columns:
    print(f"> {column}")

Columns necessary:
> ride_id
> rideable_type
> started_at
> ended_at
> start_station_name
> start_station_id
> end_station_name
> end_station_id
> start_lat
> start_lng
> end_lat
> end_lng
> member_casual


In [47]:
df_station_start_end_nulls = df_station_start_end_nulls[['ride_id', 'rideable_type', 'started_at', 'ended_at' , 'station', 'id', 'station_start_end', 'id_start_end' , 'lat', 'long' , 'lat_start_end', 'long_start_end' , 'member_casual' ]]

In [48]:
df_station_start_end_nulls.columns = ['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual']

In [49]:
len(df_station_start_end_nulls)

0

In [50]:
df_station_start_nulls = df_station_start_nulls[[
    'ride_id', 'rideable_type', 'started_at', 'ended_at' , 'station', 'id', 'end_station_name',
    'end_station_id' , 'lat', 'long' , 'end_lat', 'end_lng' , 'member_casual' ]]

In [51]:
df_station_start_nulls.columns = ['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual']

In [52]:
len(df_station_start_nulls)

500

In [53]:
df_station_end_nulls = df_station_end_nulls[['ride_id', 'rideable_type', 'started_at', 'ended_at',
                                             'start_station_name', 'start_station_id' , 'station', 'id',
                                             'start_lat', 'start_lng' , 'lat', 'long' , 'member_casual']]

In [54]:
df_station_end_nulls.columns = ['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual']

In [55]:
len(df_station_end_nulls)

0

In [56]:
df_station_end_nulls

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual


Dados juntos e com colunas já ajustadas:

In [57]:
df_station_resolved = pd.concat( [df_station_start_end_nulls , df_station_start_nulls , df_station_end_nulls] , ignore_index=True )

In [58]:
df_station_resolved.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,75F20FDAB4AD3A86,electric_bike,2020-08-10 21:43:07,2020-08-10 22:10:06,Shore Dr & 55th St,247.0,Shore Dr & 55th St,247.0,41.7952,-87.5809,41.795244,-87.580922,casual
1,2A61BE2204BD69B1,electric_bike,2020-08-22 12:00:35,2020-08-22 12:28:17,Kedzie Ave & Palmer Ct,290.0,Kedzie Ave & Palmer Ct,290.0,41.9215,-87.7073,41.921488,-87.707335,casual
2,DC2912CCFB0ACB5E,electric_bike,2020-08-31 09:04:44,2020-08-31 09:16:16,Kedzie Ave & Foster Ave,470.0,Kedzie Ave & Foster Ave,470.0,41.9757,-87.7093,41.975710,-87.709274,casual
3,251151100BEAB319,electric_bike,2020-08-20 19:28:23,2020-08-20 20:37:20,Milwaukee Ave & Rockwell St,222.0,Milwaukee Ave & Rockwell St,222.0,41.9203,-87.6926,41.920269,-87.692635,casual
4,9951CD95EDF79469,electric_bike,2020-08-13 20:04:48,2020-08-13 20:57:02,Clark St & Lincoln Ave,141.0,Clark St & Lincoln Ave,141.0,41.9157,-87.6346,41.915725,-87.634594,casual


In [59]:
print(f"Quantity of recovery data (NaN locations): {len(df_station_resolved)} lines")

Quantity of recovery data (NaN locations): 500 lines


#### Juntando os dados recuperados com os dados not null

In [60]:
df_resolved = df_not_null.copy()

In [61]:
len(df_resolved)

608510

In [62]:
df_resolved = pd.concat([df_resolved , df_station_resolved] , ignore_index=True)

In [63]:
len(df_resolved)

609010

In [64]:
percent_lost_data = ((len(df) - len(df_resolved))/len(df))*100
print(f"Lost data = {percent_lost_data} %")

Lost data = 2.1452179683495594 %


In [65]:
print(f"Previously lost data: {percent_removed_lines} %")

Previously lost data: 2.276215674352106 %


In [66]:
print(f"Minus {percent_removed_lines - percent_lost_data} %")

Minus 0.13099770600254645 %


### Com um DF completo (com os dados que conseguimos extrair) vamos iniciar a criação de algumas colunas novas)

relembrando , temos 2 df's:
> df_resolved
> 
> df_stations

In [67]:
df_resolved.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual'],
      dtype='str')

In [68]:
df_resolved.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,322BD23D287743ED,docked_bike,2020-08-20 18:08:14,2020-08-20 18:17:51,Lake Shore Dr & Diversey Pkwy,329.0,Clark St & Lincoln Ave,141.0,41.932588,-87.636427,41.915689,-87.634600,member
1,2A3AEF1AB9054D8B,electric_bike,2020-08-27 18:46:04,2020-08-27 19:54:51,Michigan Ave & 14th St,168.0,Michigan Ave & 14th St,168.0,41.864379,-87.623681,41.864221,-87.623439,casual
2,67DC1D133E8B5816,electric_bike,2020-08-26 19:44:14,2020-08-26 21:53:07,Columbus Dr & Randolph St,195.0,State St & Randolph St,44.0,41.884641,-87.619549,41.884971,-87.627574,casual
3,C79FBBD412E578A7,electric_bike,2020-08-27 12:05:41,2020-08-27 12:53:45,Daley Center Plaza,81.0,State St & Kinzie St,47.0,41.884093,-87.629639,41.889583,-87.627540,casual
4,13814D3D661ECADB,electric_bike,2020-08-27 16:49:02,2020-08-27 16:59:49,Leavitt St & Division St,658.0,Leavitt St & Division St,658.0,41.902989,-87.683767,41.903002,-87.683844,casual


In [69]:
df_resolved.isna().sum()

ride_id               0
rideable_type         0
started_at            0
ended_at              0
start_station_name    0
start_station_id      0
end_station_name      0
end_station_id        0
start_lat             0
start_lng             0
end_lat               0
end_lng               0
member_casual         0
dtype: int64

In [70]:
df_resolved.to_csv("../data/processed/dataset_not_null.csv" , index=False)

In [71]:
df_stations.to_csv("../data/processed/stations.csv" , index=False)

Algumas colunas que eu vou criar:

* round_trip = o ponto de início é o mesmo do ponto de final?
* distance = Distancia euclidiana perconrrida entre o início e o fim
* ride_duration = Tempo que o cliente ficou com a bicicleta
* is_weekend = é fim de semana
* week_day = dia da semana
* hour_start = hora de início da corrida
* hour_end = hora de fim da corrida

In [72]:
df_resolved['round_trip'] = np.where(df_resolved['start_station_name'] == df_resolved['end_station_name'], 'same_place', 'different_place')

In [73]:
df_resolved.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,round_trip
0,322BD23D287743ED,docked_bike,2020-08-20 18:08:14,2020-08-20 18:17:51,Lake Shore Dr & Diversey Pkwy,329.0,Clark St & Lincoln Ave,141.0,41.932588,-87.636427,41.915689,-87.634600,member,different_place
1,2A3AEF1AB9054D8B,electric_bike,2020-08-27 18:46:04,2020-08-27 19:54:51,Michigan Ave & 14th St,168.0,Michigan Ave & 14th St,168.0,41.864379,-87.623681,41.864221,-87.623439,casual,same_place
2,67DC1D133E8B5816,electric_bike,2020-08-26 19:44:14,2020-08-26 21:53:07,Columbus Dr & Randolph St,195.0,State St & Randolph St,44.0,41.884641,-87.619549,41.884971,-87.627574,casual,different_place
3,C79FBBD412E578A7,electric_bike,2020-08-27 12:05:41,2020-08-27 12:53:45,Daley Center Plaza,81.0,State St & Kinzie St,47.0,41.884093,-87.629639,41.889583,-87.627540,casual,different_place
4,13814D3D661ECADB,electric_bike,2020-08-27 16:49:02,2020-08-27 16:59:49,Leavitt St & Division St,658.0,Leavitt St & Division St,658.0,41.902989,-87.683767,41.903002,-87.683844,casual,same_place


In [74]:
df_resolved['ended_at'] = pd.to_datetime(df_resolved['ended_at'])
df_resolved['started_at'] = pd.to_datetime(df_resolved['started_at'])

In [75]:
df_resolved['ride_duration'] = df_resolved['ended_at'] - df_resolved['started_at']

In [76]:
df_resolved['ride_duration_minutes'] = df_resolved['ride_duration'].dt.total_seconds()/60

In [77]:
df_resolved["week_day"] = df_resolved['started_at'].dt.day_name()

In [78]:
df_resolved["is_weekend"] = df_resolved['started_at'].dt.dayofweek >= 5

In [79]:
df_resolved['hour_start'] = df_resolved['started_at'].dt.hour
df_resolved['hour_end'] = df_resolved['ended_at'].dt.hour

In [80]:
import numpy as np

def haversine_vectorized(df):
    # Raio da Terra em km
    R = 6371.0
    
    # Converte graus para radianos
    lat1, lon1 = np.radians(df['start_lat']), np.radians(df['start_lng'])
    lat2, lon2 = np.radians(df['end_lat']), np.radians(df['end_lng'])
    
    # Diferenças
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Fórmula de Haversine
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    return R * c

a Fórmula de haversine assume que a terra é de fato um globo , portanto é melhor que a distância euclidiana , visto que esta assume a terra plana (mesmo que as distâncias sejam pequenas o suficiente para não fazer diferença)

In [81]:
df_resolved['distance_km'] = haversine_vectorized(df)
df_resolved['distance_m'] = haversine_vectorized(df)*1000

In [82]:
df_resolved.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,member_casual,round_trip,ride_duration,ride_duration_minutes,week_day,is_weekend,hour_start,hour_end,distance_km,distance_m
0,322BD23D287743ED,docked_bike,2020-08-20 18:08:14,2020-08-20 18:17:51,Lake Shore Dr & Diversey Pkwy,329.0,Clark St & Lincoln Ave,141.0,41.932588,-87.636427,...,member,different_place,0 days 00:09:37,9.616667,Thursday,False,18,18,1.885153,1885.152542
1,2A3AEF1AB9054D8B,electric_bike,2020-08-27 18:46:04,2020-08-27 19:54:51,Michigan Ave & 14th St,168.0,Michigan Ave & 14th St,168.0,41.864379,-87.623681,...,casual,same_place,0 days 01:08:47,68.783333,Thursday,False,18,19,0.026647,26.647146
2,67DC1D133E8B5816,electric_bike,2020-08-26 19:44:14,2020-08-26 21:53:07,Columbus Dr & Randolph St,195.0,State St & Randolph St,44.0,41.884641,-87.619549,...,casual,different_place,0 days 02:08:53,128.883333,Wednesday,False,19,21,0.665325,665.324586
3,C79FBBD412E578A7,electric_bike,2020-08-27 12:05:41,2020-08-27 12:53:45,Daley Center Plaza,81.0,State St & Kinzie St,47.0,41.884093,-87.629639,...,casual,different_place,0 days 00:48:04,48.066667,Thursday,False,12,12,0.634686,634.686478
4,13814D3D661ECADB,electric_bike,2020-08-27 16:49:02,2020-08-27 16:59:49,Leavitt St & Division St,658.0,Leavitt St & Division St,658.0,41.902989,-87.683767,...,casual,same_place,0 days 00:10:47,10.783333,Thursday,False,16,16,0.006598,6.597641


In [83]:
df_resolved.to_csv("../data/processed/dataset_not_null_with_features.csv" , index=False)